In [1]:
import cv2 as cv
import os
import numpy as np
import json

In [2]:
import os
from scipy.ndimage import gaussian_filter
import numpy as np 
import pandas as pd 
import time 
from PIL import Image
import cv2
import pickle 
import matplotlib.pyplot as plt 
from skimage.feature import hog

In [3]:
import tensorflow as tf
import keras

Using TensorFlow backend.


In [4]:
path_dir = os.getcwd()
path_dir

'i:\\WPI\\Fall 2020\\Computer Vision - RBE 549\\CV Project\\Dataset\\cars_train\\Dimensionality Experiments'

In [5]:
fold_dir = '\\'.join((path_dir, "Images"))
car_names = os.listdir(fold_dir)

In [6]:
#Without Resizing:
start_time = time.time()

img_cars = []
for file_n in car_names:
    file_name = "\\".join((fold_dir, file_n))
    img = np.array(Image.open(file_name).convert('LA'))
    #img_grey = tf.image.resize(img, (256,256),preserve_aspect_ratio=False).numpy()
    img_cars.append(img[:,:,0])

print(time.time() - start_time)

1.4142184257507324


In [7]:
#For Image Resizing:
start_time = time.time()

img_cars_resized = []
for file_n in car_names:
    file_name = "\\".join((fold_dir, file_n))
    img = np.array(Image.open(file_name).convert('LA'))
    img_grey = cv2.resize(img, (256,256)) 
    img_cars_resized.append(img_grey[:,:,0])

print(time.time() - start_time)

1.4281504154205322


In [8]:
np.shape(img_cars)

(100,)

In [9]:
start_time = time.time()
orb = cv.ORB_create()
descriptors_normal = []
kp_normal = []
for img in img_cars:
    kp, imageDesc = orb.detectAndCompute(img, None)
    if not imageDesc is None:   
        descriptors_normal.append(imageDesc)
    if not kp is None:
        kp_normal.append(kp)

print(time.time() - start_time)

2.284975528717041


In [10]:
start_time = time.time()
orb = cv.ORB_create()
descriptors_scaled = []
kp_scaled = []
for img in img_cars_resized:
    kp, imageDesc = orb.detectAndCompute(img, None)
    if not imageDesc is None:    
        descriptors_scaled.append(imageDesc)
    if not kp is None:
        kp_scaled.append(kp)

print(time.time() - start_time)

0.4498312473297119


In [11]:
#Key points are useless!
kp_scaled[0][:5]

[<KeyPoint 0000024F3CE38ED0>,
 <KeyPoint 0000024F3CE38C00>,
 <KeyPoint 0000024F3CE385D0>,
 <KeyPoint 0000024F3CE820F0>,
 <KeyPoint 0000024F3CE821E0>]

In [19]:
flatten_original = []
normal_shape = []
flatten_normal_shape = []
for desc in descriptors_normal:
    flatten_original.append(np.resize(desc, (-1)))
    normal_shape.append(np.shape(desc)[0])
    flatten_normal_shape.append(np.shape(np.resize(desc, (-1))))

In [20]:
flatten_scaled = []
scaled_shape = []
flatten_scaled_shape = []
for desc in descriptors_scaled:
    flatten_scaled.append(np.resize(desc, (-1)))
    scaled_shape.append(np.shape(desc)[0])
    flatten_scaled_shape.append(np.shape(np.resize(desc, (-1))))

In [35]:
count, values = np.unique(normal_shape, return_counts=True)
print("Maximum Number of Descriptors - Original: ", np.max(normal_shape))
print("Minimum Number of Descriptors - Original: ",np.min(normal_shape))
print()
print(count)
print(values)

Maximum Number of Descriptors - Original:  500
Minimum Number of Descriptors - Original:  193

[193 271 368 376 388 391 393 411 413 439 462 463 477 485 490 492 494 500]
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2  1 81]


In [33]:
count, values = np.unique(scaled_shape, return_counts=True)
print("Maximum Number of Descriptors - Scaled: ", np.max(scaled_shape))
print("Minimum Number of Descriptors - Scaled: ", np.min(scaled_shape))
print()
print(count)
print(values)

Maximum Number of Descriptors - Scaled:  471
Minimum Number of Descriptors - Scaled:  381

[381 389 392 393 402 408 409 410 411 412 414 415 417 418 420 422 424 426
 427 428 429 430 431 432 433 434 435 436 437 439 440 441 442 443 444 445
 446 447 448 449 450 451 453 454 455 456 457 458 461 463 469 471]
[1 1 1 1 1 1 1 1 2 1 1 2 3 1 1 3 1 4 2 1 1 1 3 3 2 2 1 2 1 1 3 2 3 1 1 4 3
 2 1 1 2 5 7 2 1 4 4 2 2 1 1 1]


In [28]:
count, values = np.unique(flatten_scaled_shape, return_counts=True)
print("Maximum Number of Features - Scaled-Flatten: ", np.max(flatten_scaled_shape))
print("Minimum Number of Features - Scaled-Flatten: ", np.min(flatten_scaled_shape))
print()
print(count)
print(values)

Maximum Number of Features - Scaled-Flatten:  15071
Minimum Number of Features - Scaled-Flatten:  12191

[12191 12447 12543 12575 12863 13055 13087 13119 13151 13183 13247 13279
 13343 13375 13439 13503 13567 13631 13663 13695 13727 13759 13791 13823
 13855 13887 13919 13951 13983 14047 14079 14111 14143 14175 14207 14239
 14271 14303 14335 14367 14399 14431 14495 14527 14559 14591 14623 14655
 14751 14815 15007 15071]
[1 1 1 1 1 1 1 1 2 1 1 2 3 1 1 3 1 4 2 1 1 1 3 3 2 2 1 2 1 1 3 2 3 1 1 4 3
 2 1 1 2 5 7 2 1 4 4 2 2 1 1 1]


In [32]:
count, values = np.unique(flatten_normal_shape, return_counts=True)
print("Maximum Number of Features - Original-Flatten: ",np.max(flatten_normal_shape))
print("Minimum Number of Features - Original-Flatten: ",np.min(flatten_normal_shape))
print()
print(count)
print(values)

Maximum Number of Features - Original-Flatten:  15999
Minimum Number of Features - Original-Flatten:  6175

[ 6175  8671 11775 12031 12415 12511 12575 13151 13215 14047 14783 14815
 15263 15519 15679 15743 15807 15999]
[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2  1 81]
